In [1]:
import copy
from cProfile import label

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader


In [77]:
from pylab import *
import matplotlib.pyplot as plt
import os
import pyAgrum as gum
import pickle

import pyAgrum.lib.notebook as gnb

print('imported')

bn=gum.BayesNet('Causal Graph')


# high_dim=128
high_dim=20


all_nodes=['U', 'Z', 'X', 'Y']


dim={
    'U':3,
    'Z':2,
    'X': high_dim,
    'Y':2
}

U,= [bn.add(name, dim['U']) for name in "".join(['U']) ]
Z,= [bn.add(name, dim['Z']) for name in "".join(['Z']) ]
X,= [bn.add(name, dim['X']) for name in "".join(['X']) ]
Y, =[bn.add(name, dim['Y']) for name in "".join(['Y']) ]



print (bn)


bn.addArc(U,Z)
bn.addArc(U,Y)
# 
bn.addArc(Z,X)
bn.addArc(X,Y)

gnb.flow.row(bn)


# gnb.sideBySide(gnb.getBN(bn))

imported
BN{nodes: 4, arcs: 0, domainSize: 240, dim: 23, mem: 216o}


In [78]:
bn.names()

{'U', 'X', 'Y', 'Z'}

# Generate From dirilichlet distribution

# True intervention

In [79]:



# intervened graph         
def get_bn(bn2, intervened):
    for var in intervened:
        for parent in bn2.cpt(var).var_names:
            if parent != var:
                bn2.eraseArc(*(parent, var))

        
        lst = [0 for i in range(dim[var])]
        lst[intervened[var]] = 1
        bn2.cpt(var).fillWith(lst)

    return bn2




In [80]:
# Ground True intervention     

def get_true_causal_effect(bn, treatment='X', outcome='Y'):

    true_intv=[]
    for x in range(dim[treatment]):
        intervened={treatment:x}
        bn_intv=gum.BayesNet(bn)
        bn_intv= get_bn(bn_intv, intervened)
        
        all_nodes=bn_intv.names()
        ie = gum.LazyPropagation(bn_intv)
        var_set = set(all_nodes)
        ie.addJointTarget(var_set)
        ie.makeInference()
        
        pY_dox= ie.evidenceJointImpact([outcome], [])
        
        true_intv+=  list(pY_dox.toarray())
        # gnb.flow.row(pY_dox, captions=[f'P({outcome}|do{intervened})'])
    
    return true_intv


In [81]:

def powerset(s):
    
    lst=[]
    x = len(s)
    for i in range(1 << x):
        lst.append([s[j] for j in range(x) if (i & (1 << j))])

    return lst
    
# all_adj
# powerset(all_adj)

In [62]:
import math
import numpy as np
from scipy.special import xlogy
import scipy.stats as stats
import matplotlib.pyplot as plt

## set seed
# np.random.seed(0)


while(True):


    ## Gernerate distributions according to the graphical model
    dir_parm= 0.5 # initially 0.1
    
    
    pu = stats.dirichlet.rvs([dir_parm]* dim['U'], size=1)
    pz = stats.dirichlet.rvs([dir_parm]*dim['Z'], size=dim['U'])
    
    px = stats.dirichlet.rvs([dir_parm] * dim['X'], size=dim['Z'])
    py = stats.dirichlet.rvs([dir_parm] * dim['Y'], size=dim['U']*dim['X'])
    
    
    pu= np.squeeze(pu, axis=None)
    bn.cpt('U').fillWith(pu) 
    
    
    bn.cpt('Z')[:]= pz.reshape((dim['U'], dim['Z']))
    bn.cpt('Z')
    
    
    bn.cpt('X')[:]= px.reshape((dim['Z'], dim['X']))
    bn.cpt('X')
    
    
    
    bn.cpt('Y')[:]= py.reshape(( dim['X'], dim['U'], dim['Y'] ))
    bn.cpt('Y')
    
    
    # Joint distribution
    ie=gum.LazyPropagation(bn)
    ie.makeInference()
    
    
    py_z= ie.evidenceJointImpact(['Y'], ['Z'])
    # gnb.flow.row(ie.evidenceJointImpact(['Y'], ['Z']), captions=[f'P(y|z)'])
    
    
    
    # True causal effect
    true_intv = get_true_causal_effect(bn, treatment='Z', outcome='Y')
    
    
    
    
    sum=0
    for iter, value in enumerate(true_intv):
        sum+=  abs(np.array(py_z.toarray()).reshape(1,-1)[0][iter] - true_intv[iter])
    
    print(sum)
    
    if sum/2 >0.40:
        break

0.13629818450152498
0.5198652103050364
0.0016383199252369002
0.4626481656595369
0.11105737094766832
0.17778876101271457
0.7895076124336138
0.5538941916184275
0.1189843435008332
0.018127344020360675
0.28195492129830135
1.0233538733484506


In [83]:
ie=gum.LazyPropagation(bn)
ie.makeInference()
py_z= ie.evidenceJointImpact(['Y'], ['Z'])
print(py_z)

true_intv = get_true_causal_effect(bn, treatment='Z', outcome='Y')
print(true_intv)


      ||  Y                |
Z     ||0        |1        |
------||---------|---------|
0     || 0.7313  | 0.2687  |
1     || 0.2303  | 0.7697  |

[0.7111775652066921, 0.28882243479330794, 0.6422892370679784, 0.35771076293202175]


/local/scratch/a/rahman89/PycharmProjects/common-env/venv/lib/python3.10/site-packages/pyAgrum/pyAgrum.py:10018: UserWarning: 
** pyAgrum.Potential.var_names is obsolete in pyAgrum>0.22.9. Please use pyAgrum.Potential.names.

  warnings.warn("\n** pyAgrum.Potential.var_names is obsolete in pyAgrum>0.22.9. Please use pyAgrum.Potential.names.\n")


In [64]:
bn.cpt('X')

(pyAgrum.Potential<double>@0x557aba26c3e0) 
      ||  X                |
Z     ||0        |1        |
------||---------|---------|
0     || 0.3832  | 0.6168  |
1     || 0.1761  | 0.8239  |

In [65]:
import pickle

bnid=high_dim

save= f'./bn_{bnid}.pickle'
with open(save, 'wb') as handle:
    pickle.dump(bn, handle, protocol=pickle.HIGHEST_PROTOCOL)
save

'./bn_2.pickle'

In [66]:
bn

(pyAgrum.BayesNet<double>@0x557aba7233b0) BN{nodes: 4, arcs: 4, domainSize: 24, dim: 13, mem: 200o}

In [82]:
import pickle
# bnid='fedsynthetic'     #  when x is 20 dimensional.

# bnid='x20'     #  when x is 20 dimensional.
bnid=high_dim     #  when x is 64 dimensional.
with open(f'./bn_{bnid}.pickle', 'rb') as handle:
    bn = pickle.load(handle)
    
# bn.cpt('Y')
bnid

20

In [69]:
nC = 5 # number of clients

samples_per_client= 1000

g=gum.BNDatabaseGenerator(bn)
N= (nC)*samples_per_client
g.drawSamples(N,{})
main_dataset= g.to_pandas().astype(int)
main_dataset

,U,Z,X,Y
0,2,1,0,0
1,2,0,1,0
2,0,1,1,0
3,2,0,0,0
4,2,0,0,0
...,...,...,...,...
4995,0,1,1,0
4996,2,0,0,0
4997,0,1,0,1
4998,2,0,1,0


In [70]:
del main_dataset['U']
main_dataset

,Z,X,Y
0,1,0,0
1,0,1,0
2,1,1,0
3,0,0,0
4,0,0,0
...,...,...,...
4995,1,1,0
4996,0,0,0
4997,1,0,1
4998,0,1,0


# Generating client data 

In [71]:
data_chunks = np.array_split(main_dataset, nC)  # n chunks for n clients



In [72]:
project_root = '/local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM'
exp_name=f'data/frontdoor{high_dim}'
save_folder = f'{project_root}/{exp_name}'
os.makedirs(save_folder, exist_ok=True)


In [73]:
for client_no in range(nC):
    save_loc = f'{save_folder}/client{client_no}_data.pkl'
    print('Saving labels at', save_loc)
    with open(save_loc, 'wb') as f:
        pickle.dump(data_chunks[client_no], f)
    
    
    save_loc = f'{save_folder}/client{client_no}_data.csv'        
    data_chunks[client_no].to_csv(save_loc, index=False)


Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/frontdoor2/client0_data.pkl
Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/frontdoor2/client1_data.pkl
Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/frontdoor2/client2_data.pkl
Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/frontdoor2/client3_data.pkl
Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/frontdoor2/client4_data.pkl


In [74]:
data_chunks[4]

,Z,X,Y
4000,0,1,0
4001,0,0,0
4002,1,1,0
4003,1,1,0
4004,0,1,0
...,...,...,...
4995,1,1,0
4996,0,0,0
4997,1,0,1
4998,0,1,0


# Creating test dataset

In [75]:
g=gum.BNDatabaseGenerator(bn)
num_test_samples= 1000
N= num_test_samples
g.drawSamples(N,{})
test_dataset= g.to_pandas()
del test_dataset['U']
test_dataset

,Z,X,Y
0,0,0,0
1,0,1,0
2,1,1,0
3,1,1,0
4,1,1,0
...,...,...,...
995,1,1,1
996,1,1,1
997,0,0,0
998,1,1,1


In [76]:
save_loc=  f'{save_folder}/test_data.pkl'

print('Saving labels at', save_loc)
with open(save_loc, 'wb') as f:
    pickle.dump(test_dataset, f)
    
save_loc = f'{save_folder}/test_data.csv'        
data_chunks[client_no].to_csv(save_loc, index=False)

Saving labels at /local/scratch/a/rahman89/PycharmProjects/GenUnseen/MyProjects/CausalFed/FedDCM/data/frontdoor2/test_data.pkl
